In [1]:
import sys
import os
from PyQt5.QtWidgets import QApplication, QWidget, QLCDNumber
from PyQt5.QtCore import Qt, QTimer
from PyQt5.QtGui import QFont
import ctypes
import configparser

class OverlayTimer(QWidget):
    def __init__(self):
        super().__init__()
        self.initUI()

    def initUI(self):
        # 화면 크기 가져오기
        user32 = ctypes.windll.user32
        self.screen_width = user32.GetSystemMetrics(0)
        self.screen_height = user32.GetSystemMetrics(1)
        # 타이머 크기 설정 (화면 크기에 비례, 이전보다 절반 크기)
        self.timer_width = int(self.screen_width * 0.05)
        self.timer_height = int(self.screen_height * 0.025)

        # INI 파일에서 위치 불러오기 또는 기본값 설정
        self.config = configparser.ConfigParser()
        self.ini_path = 'timer_config.ini'
        if os.path.exists(self.ini_path):
            self.config.read(self.ini_path)
            pos_x = self.config.getfloat('Position', 'x', fallback=0.9) * self.screen_width
            pos_y = self.config.getfloat('Position', 'y', fallback=0.05) * self.screen_height
        else:
            pos_x = self.screen_width * 0.9
            pos_y = self.screen_height * 0.05
            self.save_position()
        # 위젯 설정
        self.setGeometry(int(pos_x), int(pos_y), self.timer_width, self.timer_height)
        self.setWindowTitle('PoE Timer')
        self.setWindowFlags(Qt.WindowStaysOnTopHint | Qt.FramelessWindowHint | Qt.Tool)
        self.setAttribute(Qt.WA_TranslucentBackground)

        # LCD 숫자 디스플레이 설정
        self.lcd = QLCDNumber(self)
        self.lcd.setDigitCount(8)
        self.lcd.setSegmentStyle(QLCDNumber.Flat)
        self.lcd.setStyleSheet("background-color: rgba(0, 0, 0, 100); color: rgb(255, 0, 0);")
        self.lcd.resize(self.timer_width, self.timer_height)

        # 타이머 설정
        self.timer = QTimer(self)
        self.timer.timeout.connect(self.showTime)
        self.timer.start(1000)

        self.seconds = 0
        self.showTime()

    def showTime(self):
        self.seconds += 1
        m, s = divmod(self.seconds, 60)
        h, m = divmod(m, 60)
        time = f"{h:02d}:{m:02d}:{s:02d}"
        self.lcd.display(time)

    def mousePressEvent(self, event):
        if event.button() == Qt.LeftButton:
            self.dragPosition = event.globalPos() - self.frameGeometry().topLeft()
            event.accept()

    def mouseMoveEvent(self, event):
        if event.buttons() == Qt.LeftButton:
            self.move(event.globalPos() - self.dragPosition)
            event.accept()

    def mouseReleaseEvent(self, event):
        if event.button() == Qt.LeftButton:
            self.save_position()

    def save_position(self):
        x = self.x() / self.screen_width
        y = self.y() / self.screen_height
        self.config['Position'] = {'x': str(x), 'y': str(y)}
        with open(self.ini_path, 'w') as configfile:
            self.config.write(configfile)
if __name__ == '__main__':
    app = QApplication(sys.argv)
    ex = OverlayTimer()
    ex.show()
    sys.exit(app.exec_())


KeyboardInterrupt: 